# Spatial-Temporal Experiment

In [1]:
import sys, os
cwd = os.getcwd()
sys.path.insert(0, f'{cwd}/../../')
sys.path.insert(0, '/home/emmanuel/code/py_esdc')
sys.path.insert(0, '/home/emmanuel/code/gp_model_zoo')

# standard python packages
import xarray as xr
import pandas as pd
import numpy as np

# 
from src.models.spatemp.train_models import Metrics

# esdc tools
from esdc.subset import select_pixel
from esdc.shape import ShapeFileExtract, rasterize
from esdc.transform import DensityCubes

import matplotlib.pyplot as plt
import cartopy
import cartopy.crs as ccrs
plt.style.use(['fivethirtyeight', 'seaborn-poster'])
%matplotlib inline

%load_ext autoreload
%autoreload 2

## 1. Get DataCubes

In [2]:
filename = '/media/disk/databases/ESDC/esdc-8d-0.25deg-1x720x1440-2.0.0.zarr'

datacube = xr.open_zarr(filename)

In [3]:
lst_cube = datacube.soil_moisture
lst_cube

<xarray.DataArray 'soil_moisture' (time: 1702, lat: 720, lon: 1440)>
dask.array<zarr, shape=(1702, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 89.875 89.625 89.375 ... -89.375 -89.625 -89.875
  * lon      (lon) float32 -179.875 -179.625 -179.375 ... 179.625 179.875
  * time     (time) datetime64[ns] 1980-01-05 1980-01-13 ... 2016-12-30
Attributes:
    ID:                        78
    esa_cci_path:              /neodc/esacci/soil_moisture/data/daily_files/C...
    long_name:                 Soil Moisture
    orig_attrs:                {'comment': 'Soil moisture based on the SOilmo...
    orig_version:              v04.2
    project_name:              SoilMoisture CCI
    time_coverage_end:         2014-01-29
    time_coverage_resolution:  P8D
    time_coverage_start:       1980-01-05
    url:                       http://www.esa-soilmoisture-cci.org

## 2. Select Region

In [4]:
europe = lst_cube.sel(lat=slice(71.5, 35.5), lon=slice(-18.0, 60.0))

## 3. Get Density Cubes

In [5]:
spatial = 1
temporal = 46

# initialize minicuber
minicuber = DensityCubes(
    spatial_window=spatial, 
    time_window=temporal, 
)

europe_df = minicuber.get_minicubes(europe)

In [6]:
europe_df.shape

(2052734, 46)

## 4. ML Model Framework

### 4.1 Preprocessing

#### 4.1.1 - Training and testing

In [7]:
europe_df.head()


var_x0    var_x1    var_x2    var_x3    var_x4  \
time       lat    lon                                                         
1986-04-03 41.375  59.125  0.274000  0.169400  0.148700  0.202800  0.141000   
2002-09-26 52.375 -7.625   0.269040  0.277484  0.284080  0.304824  0.293366   
                  -7.375   0.250482  0.277345  0.244725  0.315922  0.293825   
           52.125 -8.625   0.247233  0.263234  0.251600  0.259300  0.269975   
           51.875 -9.125   0.270028  0.239867  0.243008  0.274140  0.264556   

                             var_x5    var_x6    var_x7    var_x8    var_x9  \
time       lat    lon                                                         
1986-04-03 41.375  59.125  0.112900  0.119000  0.131900  0.119000  0.121850   
2002-09-26 52.375 -7.625   0.293524  0.289478  0.285967  0.290185  0.308285   
                  -7.375   0.302100  0.285718  0.303325  0.283058  0.305284   
           52.125 -8.625   0.254725  0.263375  0.258550  0.299240  0.283700   
           51.875 -9.125   0.270744  0.277584  0.287440  0.289464  0.291867   

                           ...   var_x36   var_x37  var_x38   var_x39  \
time       lat    lon      ...                                          
1986-04-03 41.375  59.125  ...  0.230600  0.230600  0.21670  0.216700   
2002-09-26 52.375 -7.625   ...  0.262700  0.257650  0.30050  0.277167   
                  -7.375   ...  0.220200  0.257333  0.25950  0.229800   
           52.125 -8.625   ...  0.247500  0.240150  0.27155  0.263400   
           51.875 -9.125   ...  0.236344  0.232978  0.27610  0.240824   

                            var_x40   var_x41   var_x42   var_x43   var_x44  \
time       lat    lon                                                         
1986-04-03 41.375  59.125  0.133300  0.202800  0.229056  0.202800  0.216700   
2002-09-26 52.375 -7.625   0.252000  0.263375  0.264067  0.228867  0.250912   
                  -7.375   0.224600  0.225178  0.243500  0.260350  0.220200   
           52.125 -8.625   0.245900  0.240825  0.233130  0.245700  0.237628   
           51.875 -9.125   0.244157  0.210883  0.235125  0.250400  0.228756   

                            var_x45  
time       lat    lon                
1986-04-03 41.375  59.125  0.216700  
2002-09-26 52.375 -7.625   0.176800  
                  -7.375   0.233022  
           52.125 -8.625   0.178400  
           51.875 -9.125   0.225722  

[5 rows x 46 columns]

In [8]:
y = europe_df.iloc[:, 0][:, np.newaxis]
X = europe_df.iloc[:, 1:]

d_dimensions = X.shape[1]

#### 4.1.2 - Train-Test Split

In [9]:
from sklearn.model_selection import train_test_split


train_size = 10_000
random_state = 123

xtrain, xtest, ytrain, ytest = train_test_split(
    X, y, train_size=train_size, random_state=random_state)

test_size = xtest.shape[0]

#### 4.1.1 - Normalize

In [10]:
from sklearn.preprocessing import StandardScaler

# normalize inputs
x_normalizer = StandardScaler(with_mean=True, with_std=False)

xtrain_norm = x_normalizer.fit_transform(xtrain)
xtest_norm = x_normalizer.transform(xtest)

# remove mean outputs
y_normalizer = StandardScaler(with_std=False)

ytrain_norm = y_normalizer.fit_transform(ytrain)
ytest_norm = y_normalizer.transform(ytest)

### 4.2 - Training

In [11]:
from gpy.sparse import SparseGPR
import GPy

In [12]:
# gp params
n_dims = xtrain_norm.shape[1]
kernel = GPy.kern.RBF(input_dim=n_dims, ARD=False)
inference = 'vfe'
n_inducing = 300
verbose = 1
max_iters = 5_000
n_restarts = 0

# initialize GP Model
sgp_model = SparseGPR(
    kernel=kernel, 
    inference=inference, 
    n_inducing=n_inducing, 
    verbose=verbose,
    max_iters=max_iters,
    n_restarts=n_restarts
)

# train GP model
sgp_model.fit(xtrain_norm, ytrain_norm)

 /home/emmanuel/.conda/envs/2019_rbig_ad/lib/python3.6/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/emmanuel/.conda/envs/2019_rbig_ad/lib/python3.6/site-packages/GPy/kern/src/stationary.py:353: RuntimeWarning:invalid value encountered in true_divide
 /home/emmanuel/.conda/envs/2019_rbig_ad/lib/python3.6/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /home/emmanuel/.conda/envs/2019_rbig_ad/lib/python3.6/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


SparseGPR(alpha=0.5, inference='vfe',
          kernel=<GPy.kern.src.rbf.RBF object at 0x7ffa8c70cb38>,
          max_iters=5000, n_inducing=300, n_restarts=0, optimizer='scg',
          verbose=1)

In [13]:
sgp_model.display_model()

sparse_gp.,value,constraints,priors
inducing inputs,"(300, 45)",,
rbf.variance,0.01917727197037847,+ve,
rbf.lengthscale,1.4028640633585876,+ve,
Gaussian_noise.variance,0.0011924646044146395,+ve,


### 4.3 - Testing

In [14]:
ypred = sgp_model.predict(xtest_norm, return_std=False)

In [15]:
ypred.shape, ytest_norm.shape

((2042734, 1), (2042734, 1))

In [16]:
stats = Metrics().get_all(ypred.squeeze(), ytest_norm.squeeze())
stats

,mae,mse,rmse,r2
0,0.025959,0.001244,0.035273,0.770773
